모델을 불러오는 과정

In [1]:
import cv2
from ultralytics import YOLO
from ultralytics.engine.results import Results

# Load a pretrained YOLOv8n model
model = YOLO('yolov8x-seg.pt')

# Run inference on an image
results: Results = model('예제선반1.jpg')  # results list


image 1/1 l:\2024-1\SW\0429   \Arrangement_of_Jinn_Dolsoe\1.jpg: 480x640 4 bottles, 1 cup, 624.0ms
Speed: 1.0ms preprocess, 624.0ms inference, 504.2ms postprocess per image at shape (1, 3, 480, 640)


선반을 관리하는 클래스

In [2]:
class Shelf:
    def __init__(self, x1, y1, x2, y2):
        self.x1 = min(x1, x2)
        self.y1 = min(y1, y2)
        self.x2 = max(x1, x2)
        self.y2 = max(y1, y2)
        self.width = self.x2 - self.x1
        self.height = self.y2 - self.y1

    def __str__(self):
        return f'x1: {self.x1}, y1: {self.y1}, x2: {self.x2}, y2: {self.y2}'
    
    def print(self):
        print(f"선반의 가로 길이: {self.width}, 선반의 세로 길이: {self.height}")

마우스 드래그로 사각형을 그리는 기능

In [4]:
# Global variables
drawing = False # true if mouse is pressed
ix, iy = -1, -1
img = None
temp_img = None
shelf_object = list()

# Mouse callback function
def draw_rectangle(event, x, y, flags, param):
    global ix, iy, drawing, img, temp_img

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix, iy = x, y
        temp_img = img.copy()

    elif event == cv2.EVENT_MOUSEMOVE:  
        if drawing is True:
            img = temp_img.copy()
            cv2.rectangle(img, (ix, iy), (x, y), (0, 255, 0), 1)

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        cv2.rectangle(img, (ix, iy), (x, y), (0, 255, 0), -1)
        shelf_object.append(Shelf(ix, iy, x, y))
        print(f"선반의 좌표: {shelf_object[-1]}")
        print(f"선반의 가로 길이: {shelf_object[-1].width}, 선반의 세로 길이: {shelf_object[-1].height}")

# Load an image
img = results[0].orig_img.copy()
cv2.namedWindow('image')
cv2.setMouseCallback('image', draw_rectangle)

while(1):
    cv2.imshow('image', img)
    if cv2.waitKey(1) & 0xFF == 27:  # exit if ESC is pressed
        break

cv2.destroyAllWindows()

선반의 좌표: x1: 100, y1: 264, x2: 1188, y2: 847
선반의 가로 길이: 1088, 선반의 세로 길이: 583


탐지된 객체를 관리하는 클래스

In [5]:
class YoloObject:
    def __init__(self, object, image, width, height):
        self.object = object
        self.image = image
        self.width = width
        self.height = height

    def show(self):
        self.object.show()

객체를 하나씩 보여주고 가로 세로 길이를 출력하는 기능

In [6]:
image_path = '예제선반1.jpg'
image = cv2.imread(image_path)
yolo_objects = list()

for index, r in enumerate(results[0]):  # 인덱스와 결과를 반복한다
    box = r.boxes.xyxy[0]  # 경계 상자 좌표 가져오기
    x1, y1, x2, y2 = map(int, box[:4])  # 경계 상자 좌표 (정수로 변환)
    cropped_img = image[y1:y2, x1:x2]  # 경계 상자 좌표를 사용하여 이미지 자르기

    height, width = cropped_img.shape[:2]
    new_width = int(width)
    new_height = int(height)
    cropped_img = cv2.resize(cropped_img, (new_width, new_height))

    yolo_objects.append(YoloObject(r, cropped_img, new_width, new_height))
        
    print(f'물건 {index} 가로 길이: {new_width}, 세로 길이: {new_height}')  # 객체의 가로와 세로 길이 출력
    cv2.imshow(f'Detected Object {index}', cropped_img)
    cv2.waitKey(0)

cv2.destroyAllWindows() # 이미지창 닫기

물건 0 가로 길이: 112, 세로 길이: 322
물건 1 가로 길이: 125, 세로 길이: 431
물건 2 가로 길이: 110, 세로 길이: 173
물건 3 가로 길이: 138, 세로 길이: 460
물건 4 가로 길이: 54, 세로 길이: 199


In [7]:
cv2.imshow("image", yolo_objects[2].image)
cv2.waitKey(0)

cv2.destroyAllWindows() # 이미지창 닫기

In [8]:
yolo_objects[2].show()

shelf_object에는 선반의 좌표가 저장되어 있고, yolo_objects에는 물건의 좌표가 저장되어 있습니다. yolo_objects의 크기를 기준으로 선반의 크기에 맞게 들어가는지 확인을 하고 들어간다면 좌측 하단부터 시작하여 오른쪽으로 이동하면서 물건을 놓을 수 있는지 확인을 합니다. 물건을 놓을 수 있다면 물건의 좌표를 yolo_objects에 추가하고, 놓을 수 없다면 다음 줄로 넘어가서 물건을 놓을 수 있는지 확인을 합니다.

In [17]:
image_path = '예제선반1.jpg'
image = cv2.imread(image_path)

image[shelf_object[0].y1:shelf_object[0].y2, shelf_object[0].x1:shelf_object[0].x2] = [0, 128, 0]

print(f"선반의 가로 길이: {shelf_object[0].width}")  # 선반의 너비를 출력한다.
print(f"선반의 세로 길이: {shelf_object[0].height}")  # 선반의 높이를 출력한다.
print(f"선반의 좌표: {shelf_object[0].x1}, {shelf_object[0].y1}, {shelf_object[0].x2}, {shelf_object[0].y2}")

index = 0  # 인덱스를 0으로 설정한다.

for obj in yolo_objects:
    if obj.width <= shelf_object[0].width - index:  # 물체의 너비가 남는 공간보다 작거나 같으면
        좌측상단x = shelf_object[0].x1 + index
        좌측상단y = shelf_object[0].y2 - obj.height
        우측하단x = shelf_object[0].x1 + index + obj.width
        우측하단y = shelf_object[0].y2
        image[좌측상단y:우측하단y, 좌측상단x:우측하단x] = obj.image
        index += obj.width  # 인덱스에 물체의 너비를 더한다.
        print(f"선반의 용량: {index}/{shelf_object[0].width}")  # 선반의 용량을 출력한다.

cv2.imshow("image", image)
cv2.waitKey(0)

cv2.destroyAllWindows() # 이미지창 닫기

선반의 가로 길이: 1088
선반의 세로 길이: 583
선반의 좌표: 100, 264, 1188, 847
선반의 용량: 112/1088
선반의 용량: 237/1088
선반의 용량: 347/1088
선반의 용량: 485/1088
선반의 용량: 539/1088
